### System settings verification

In [ ]:
import sys
sys.version

In [4]:
!which python
!echo $PYTHONPATH
!java --version

/Users/harjeet/DevEnv/miniconda3/bin/python
/usr/local/Cellar/apache-spark/2.3.0/libexec//python/lib/py4j-0.10.6-src.zip:/usr/local/Cellar/apache-spark/2.3.0/libexec//python/:
java 10.0.1 2018-04-17
Java(TM) SE Runtime Environment 18.3 (build 10.0.1+10)
Java HotSpot(TM) 64-Bit Server VM 18.3 (build 10.0.1+10, mixed mode)


# Creating a Spark Context

In [ ]:
from pyspark.sql import DataFrameReader, SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [5]:
spark = SparkSession\
    .builder\
    .config("master","local[*]")\
    .appName("LastFM")\
    .getOrCreate()

In [7]:
spark.sparkContext._conf.getAll()

[('spark.driver.host', '172.16.10.66'),
 ('master', 'local[*]'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '60501'),
 ('spark.app.id', 'local-1526481358426'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'LastFM')]

# Getting the data
Here's the [README](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-readme.txt) for a description.
Briefly:
- 92834 "scrobbles" (play counts) per user of an artist
- 1892 users
- 17632 artists
- density/sparsity of the data: 0.28%

In [10]:
import os
import wget
url = 'http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip'
filename = wget.download(url)
filename

'hetrec2011-lastfm-2k.zip'

In [ ]:
import zipfile